In [1]:
#Make sure the files available in yout google drive at folder projct_dog_files
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.23 MiB | 25.98 MiB/s, done.
Resolving deltas: 100% (511/511), done.
/content/yolov7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [2]:
#Loading custom detect.py file to replace with yolo detect.py

from google.colab import drive
drive.mount('/content/drive')
import shutil
colab_link = "/content/yolov7/"
gdrive_link = "/content/drive/MyDrive/projct_dog_files/detect.py"
shutil.copy(gdrive_link,colab_link)

Mounted at /content/drive


'/content/yolov7/detect.py'

In [3]:
#Copying video to process from google drive
filename_to_fetch_and_start='bella_dog_video_to_upload.mp4';
colab_link = "/content/"
gdrive_link = "/content/drive/MyDrive/projct_dog_files/"+filename_to_fetch_and_start
shutil.copy(gdrive_link,colab_link)

'/content/bella_dog_video_to_upload.mp4'

In [ ]:
#Running yolov7 for dog detection which will detect dogs and track movements
import uuid

unique_id_folder= uuid.uuid4()
filename_to_process_yolo='/content/'+filename_to_fetch_and_start;
!python detect.py --source $filename_to_process_yolo --weights yolov7.pt --name $unique_id_folder

In [ ]:
import json
from pathlib import Path
colab_link = "/content/"
gdrive_link = "/content/drive/MyDrive/projct_dog_files/emotion_model_new.pkl"
shutil.copy(gdrive_link,colab_link)
#Give the path of video file which we going to detect dog emotion
file_name_to_process=filename_to_process_yolo
detection_json_file_name=Path(file_name_to_process).stem
with open('/content/'+detection_json_file_name+'_detection.json') as f:
    frame_to_look = json.load(f)

import cv2
from fastai.vision.widgets import *
from fastai.vision.all import *
import  numpy as np
import json
import os
from datetime import timedelta

vidcap = cv2.VideoCapture(file_name_to_process)
learn = load_learner('/content/emotion_model_new.pkl', cpu=False)

total_video_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
success,image1 = vidcap.read()
count=0
post_frame = 0
FPS = 24.0
import uuid
detected_time_json={}

key_list_to_look=list(frame_to_look.keys())
key_list_to_look = [ int(x) for x in key_list_to_look ]

while success:

    success,image1 = vidcap.read()
    post_frame = vidcap.get(1)
    if post_frame in key_list_to_look:
      filename=str(uuid.uuid4())+'.jpg'
      cv2.imwrite('/content/'+filename, image1)

      pred=learn.predict('/content/'+filename)

      predicted_class = pred[0]
      frame_count = count
      os.remove('/content/'+filename)

      detected_time_json[str(int(post_frame))]=predicted_class
      print(str(post_frame)+'/'+str(total_video_length))


with open('/content/'+detection_json_file_name+'_emotion.json', 'w') as fp:
        json.dump(detected_time_json, fp)


In [ ]:
#converting into libx264 so browser can play the file
yolo_output_path='/content/yolov7/runs/detect/'+str(unique_id_folder)+'/'+filename_to_fetch_and_start
Path("/content/output/").mkdir(parents=True, exist_ok=True)
mp4_output_path='/content/output/'+filename_to_fetch_and_start
!ffmpeg -i $yolo_output_path -vcodec libx264 $mp4_output_path
from google.colab import files
files.download(mp4_output_path)
files.download('/content/'+detection_json_file_name+'_detection.json')
files.download('/content/'+detection_json_file_name+'_emotion.json')
files.download('/content/'+detection_json_file_name+'_result.json')